In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [32]:
# Scrape list of beer styles from beeradvocate.com
url="https://www.beeradvocate.com/beer/style/"
r = requests.get(url)
soup = BeautifulSoup(r.content,"lxml")

styles=soup.find_all(href=re.compile("/beer/style/"))
names=[]
links=[]
for style in styles:
    name=style.contents[0]
    link="https://www.beeradvocate.com"+style.get("href")
    names.append(name)
    links.append(link)

links=links[2:-1]
names=names[2:-1]
style_ids=[link.split("/")[-2] for link in links]

style_table=pd.DataFrame({"style":pd.Series(names),"link":pd.Series(links),"id":pd.Series(style_ids)})
style_table.to_csv("styles.csv", encoding="utf-8")

In [35]:
# Read pre-saved list of beer styles
styles=pd.read_csv("styles.csv", encoding="utf-8")
links=styles["link"]
names=styles["style"]
ids=styles["id"]

In [38]:
# Scrape beers of each styles from beeradvocate.com
style_id=0

while style_id<len(names):
    style_url=links[style_id]
    style_name = "style_"+str(ids[style_id])+".csv"
    r = requests.get(style_url)
    soup = BeautifulSoup(r.content,"lxml")
    print("Start parsing {}".format(names[style_id]))
    
    indexs=soup.find_all(href=re.compile(style_url[29:]+"\?sort\=revsD\&start\="))
    for ind in indexs:
        if ind.contents[0]=="last":
            last_page=int(ind.get("href").split("=")[-1])
            break
    print("Total items: {}".format(last_page))
    
    beer_names=[]
    beer_ids=[]
    companys=[]
    company_ids=[]
    abvs=[]
    ratings=[]
    scores=[]
    beer_links=[]

    page=0

    while page*50<=last_page:
        beer_url=style_url+"?sort=revsD&start="+str(page*50)
        r = requests.get(beer_url)
        soup = BeautifulSoup(r.content,"lxml")
        beers=soup.find_all(class_="hr_bottom_light")
        assert len(beers)%5==0
        for i in range(int(len(beers)/5)):
            beer_name = beers[i*5].contents[0].contents[0].contents[0]
            beer_id = beers[i*5].contents[0].get("href").split("/")[-2]
            company = beers[i*5+1].contents[0].contents[0]
            company_id = beers[i*5].contents[0].get("href").split("/")[-3]
            abv = beers[i*5+2].contents[0].contents[0]
            rating = beers[i*5+3].contents[0].contents[0]
            score = beers[i*5+4].contents[0].contents[0]
            beer_link="https://www.beeradvocate.com/beer/profile/"+company_id+"/"+beer_id+"/"

            beer_names.append(beer_name)
            beer_ids.append(beer_id)
            companys.append(company)
            company_ids.append(company_id)
            abvs.append(abv)
            ratings.append(rating)
            scores.append(score)
            beer_links.append(beer_link)

        page+=1

    df=pd.DataFrame({"beer_names":pd.Series(beer_names),"beer_ids":pd.Series(beer_ids),"companys":pd.Series(companys),
                     "company_ids":pd.Series(company_ids),"abvs":pd.Series(abvs),"ratings":pd.Series(ratings),"scores":pd.Series(scores),
                    "beer_links":pd.Series(beer_links)})

    df.to_csv(style_name, encoding="utf-8")
    print("{} completed".format(style_name))
    print("Total beers: {}".format(df.shape[0]))
    style_id+=1

Start parsing American Amber / Red Ale
Total items: 6600
style_128.csv completed
Total beers: 6650
Start parsing American Barleywine
Total items: 1450
style_19.csv completed
Total beers: 1485
Start parsing American Black Ale
Total items: 2500
style_175.csv completed
Total beers: 2544
Start parsing American Blonde Ale
Total items: 4750
style_99.csv completed
Total beers: 4776
Start parsing American Brown Ale
Total items: 4450
style_73.csv completed
Total beers: 4475
Start parsing American Dark Wheat Ale
Total items: 200
style_94.csv completed
Total beers: 202
Start parsing American Double / Imperial IPA
Total items: 11700
style_140.csv completed
Total beers: 11730
Start parsing American Double / Imperial Stout
Total items: 5900
style_157.csv completed
Total beers: 5931
Start parsing American IPA
Total items: 29400
style_116.csv completed
Total beers: 29413
Start parsing American Pale Ale (APA)
Total items: 14350
style_97.csv completed
Total beers: 14352
Start parsing American Pale Wheat

style_131.csv completed
Total beers: 818
Start parsing Maibock / Helles Bock
Total items: 900
style_33.csv completed
Total beers: 916
Start parsing Märzen / Oktoberfest
Total items: 2450
style_29.csv completed
Total beers: 2479
Start parsing Munich Dunkel Lager
Total items: 900
style_46.csv completed
Total beers: 948
Start parsing Munich Helles Lager
Total items: 1250
style_21.csv completed
Total beers: 1263
Start parsing Rauchbier
Total items: 400
style_7.csv completed
Total beers: 436
Start parsing Schwarzbier
Total items: 1150
style_31.csv completed
Total beers: 1168
Start parsing Vienna Lager
Total items: 1000
style_30.csv completed
Total beers: 1039
Start parsing Happoshu
Total items: 50
style_168.csv completed
Total beers: 66
Start parsing Japanese Rice Lager
Total items: 100
style_169.csv completed
Total beers: 104
Start parsing Fruit / Vegetable Beer
Total items: 4600
style_9.csv completed
Total beers: 4612
Start parsing Herbed / Spiced Beer
Total items: 1750
style_8.csv comple

In [4]:
# Read pre-saved list of beer styles
styles=pd.read_csv("styles.csv", encoding="utf-8")
links=styles["link"]
names=styles["style"]
ids=styles["id"]

style_id=0

company_ids=set([])

while style_id<len(names):
    style_url=links[style_id]
    style_name = "style_"+str(ids[style_id])+".csv"
    df=pd.read_csv(style_name, encoding="utf-8")
    company_ids.update(df["company_ids"].tolist())
    style_id+=1

company_ids=list(company_ids)

In [22]:
# Scrape brewery info from beeradvocate.com 
company_urls=[]
company_names=[]
location1s=[]
location2s=[]

ba_scores=[]
ba_levels=[]
total_ratings=[]

beer_currents=[]
total_reviews=[]

In [21]:
'''
# Temperorary

df=pd.read_csv("company.csv",encoding="utf-8")

company_urls=df["company_urls"].tolist()
company_names=df["company_names"].tolist()
location1s=df["location1s"].tolist()
location2s=df["location2s"].tolist()

ba_scores=df["ba_scores"].tolist()
ba_levels=df["ba_levels"].tolist()
total_ratings=df["total_ratings"].tolist()

beer_currents=df["beer_currents"].tolist()
total_reviews=df["total_reviews"].tolist()

print(df.shape)
df.tail()

'''

(11110, 11)


,Unnamed: 0,ba_levels,ba_scores,beer_currents,company_ids,company_names,company_urls,location1s,location2s,total_ratings,total_reviews
11105,11105,NaN,0.00,7,52233,Royal Palm Brewing Company,https://www.beeradvocate.com/beer/profile/52233/,Royal Palm Beach,FL,0,0
11106,11106,Good,3.68,1,52235,Isaac Poad Brewery,https://www.beeradvocate.com/beer/profile/52235/,York,GB2,1,1
11107,11107,Exceptional,4.19,1,52237,Salty Turtle Beer Company,https://www.beeradvocate.com/beer/profile/52237/,Surf City,NC,1,1
11108,11108,NaN,0.00,3,52238,Beaches Brewing Company,https://www.beeradvocate.com/beer/profile/52238/,Toronto,ON,0,0
11109,11109,Awful,1.96,1,52240,Birrificio dei Castelli,https://www.beeradvocate.com/beer/profile/52240/,Arcevia (AN),Italy,1,1


In [22]:
for i in range(len(company_ids)):
    comp_id=company_ids[i]
    company_url="https://www.beeradvocate.com/beer/profile/"+str(comp_id)+"/"
    r = requests.get(company_url)
    soup = BeautifulSoup(r.content,"lxml")
    
    company_info=soup.find_all(attrs={"name": "keywords"}) [0].get("content")
    score_summary=soup.find_all(id="score_box")[0].contents
    score_detail=soup.find_all(id="item_stats")[0].contents[1].contents

    company_name=company_info.split(",")[0]
    try:
        location1=company_info.split(",")[1]
    except IndexError:
        location1="NA"   

    try:
        location2=company_info.split(",")[2]
    except IndexError:
        location2="NA"

    ba_score=score_summary[6].contents[0].contents[0]
    try:
        ba_level=score_summary[10].contents[0]
    except IndexError:
        ba_level="NA"
    total_rating=score_summary[-1].split("\t")[0]

    beer_current=score_detail[3].contents[0]
    total_review=score_detail[7].contents[0]
    
    company_urls.append(company_url)
    company_names.append(company_name)
    location1s.append(location1)
    location2s.append(location2)
    ba_scores.append(ba_score)
    ba_levels.append(ba_level)
    total_ratings.append(total_rating)
    beer_currents.append(beer_current)
    total_reviews.append(total_review)
    print("Finished: {}".format(i))
    


Finished: 11110
Finished: 11111
Finished: 11112
Finished: 11113
Finished: 11114
Finished: 11115
Finished: 11116
Finished: 11117
Finished: 11118
Finished: 11119
Finished: 11120
Finished: 11121
Finished: 11122
Finished: 11123
Finished: 11124
Finished: 11125
Finished: 11126
Finished: 11127
Finished: 11128
Finished: 11129
Finished: 11130
Finished: 11131
Finished: 11132
Finished: 11133
Finished: 11134
Finished: 11135
Finished: 11136
Finished: 11137
Finished: 11138
Finished: 11139
Finished: 11140
Finished: 11141
Finished: 11142
Finished: 11143
Finished: 11144
Finished: 11145
Finished: 11146
Finished: 11147
Finished: 11148
Finished: 11149
Finished: 11150
Finished: 11151
Finished: 11152
Finished: 11153
Finished: 11154
Finished: 11155
Finished: 11156
Finished: 11157
Finished: 11158
Finished: 11159
Finished: 11160
Finished: 11161
Finished: 11162
Finished: 11163
Finished: 11164
Finished: 11165
Finished: 11166
Finished: 11167
Finished: 11168
Finished: 11169
Finished: 11170
Finished: 11171
Finished

Finished: 11623
Finished: 11624
Finished: 11625
Finished: 11626
Finished: 11627
Finished: 11628
Finished: 11629
Finished: 11630
Finished: 11631
Finished: 11632
Finished: 11633
Finished: 11634
Finished: 11635
Finished: 11636
Finished: 11637
Finished: 11638
Finished: 11639
Finished: 11640
Finished: 11641
Finished: 11642
Finished: 11643
Finished: 11644
Finished: 11645
Finished: 11646
Finished: 11647
Finished: 11648
Finished: 11649
Finished: 11650
Finished: 11651
Finished: 11652
Finished: 11653
Finished: 11654
Finished: 11655
Finished: 11656
Finished: 11657
Finished: 11658
Finished: 11659
Finished: 11660
Finished: 11661
Finished: 11662
Finished: 11663
Finished: 11664
Finished: 11665
Finished: 11666
Finished: 11667
Finished: 11668
Finished: 11669
Finished: 11670
Finished: 11671
Finished: 11672
Finished: 11673
Finished: 11674
Finished: 11675
Finished: 11676
Finished: 11677
Finished: 11678
Finished: 11679
Finished: 11680
Finished: 11681
Finished: 11682
Finished: 11683
Finished: 11684
Finished

Finished: 12136
Finished: 12137
Finished: 12138
Finished: 12139
Finished: 12140
Finished: 12141
Finished: 12142
Finished: 12143
Finished: 12144
Finished: 12145
Finished: 12146
Finished: 12147
Finished: 12148
Finished: 12149
Finished: 12150
Finished: 12151
Finished: 12152
Finished: 12153
Finished: 12154
Finished: 12155
Finished: 12156
Finished: 12157
Finished: 12158
Finished: 12159
Finished: 12160
Finished: 12161
Finished: 12162
Finished: 12163
Finished: 12164
Finished: 12165
Finished: 12166
Finished: 12167
Finished: 12168
Finished: 12169
Finished: 12170
Finished: 12171
Finished: 12172
Finished: 12173
Finished: 12174
Finished: 12175
Finished: 12176
Finished: 12177
Finished: 12178
Finished: 12179
Finished: 12180
Finished: 12181
Finished: 12182
Finished: 12183
Finished: 12184
Finished: 12185
Finished: 12186
Finished: 12187
Finished: 12188
Finished: 12189
Finished: 12190
Finished: 12191
Finished: 12192
Finished: 12193
Finished: 12194
Finished: 12195
Finished: 12196
Finished: 12197
Finished

Finished: 12649
Finished: 12650
Finished: 12651
Finished: 12652
Finished: 12653
Finished: 12654
Finished: 12655
Finished: 12656
Finished: 12657
Finished: 12658
Finished: 12659
Finished: 12660
Finished: 12661
Finished: 12662
Finished: 12663
Finished: 12664
Finished: 12665
Finished: 12666
Finished: 12667
Finished: 12668
Finished: 12669
Finished: 12670
Finished: 12671
Finished: 12672
Finished: 12673
Finished: 12674
Finished: 12675
Finished: 12676
Finished: 12677
Finished: 12678
Finished: 12679
Finished: 12680
Finished: 12681
Finished: 12682
Finished: 12683
Finished: 12684
Finished: 12685
Finished: 12686
Finished: 12687
Finished: 12688
Finished: 12689
Finished: 12690
Finished: 12691
Finished: 12692
Finished: 12693
Finished: 12694
Finished: 12695
Finished: 12696
Finished: 12697
Finished: 12698
Finished: 12699
Finished: 12700
Finished: 12701
Finished: 12702
Finished: 12703
Finished: 12704
Finished: 12705
Finished: 12706
Finished: 12707
Finished: 12708
Finished: 12709
Finished: 12710
Finished

Finished: 13162
Finished: 13163
Finished: 13164
Finished: 13165
Finished: 13166
Finished: 13167
Finished: 13168
Finished: 13169
Finished: 13170
Finished: 13171
Finished: 13172
Finished: 13173
Finished: 13174
Finished: 13175
Finished: 13176
Finished: 13177
Finished: 13178
Finished: 13179
Finished: 13180
Finished: 13181
Finished: 13182
Finished: 13183
Finished: 13184
Finished: 13185
Finished: 13186
Finished: 13187
Finished: 13188
Finished: 13189
Finished: 13190
Finished: 13191
Finished: 13192
Finished: 13193
Finished: 13194
Finished: 13195
Finished: 13196
Finished: 13197
Finished: 13198
Finished: 13199
Finished: 13200
Finished: 13201
Finished: 13202
Finished: 13203
Finished: 13204
Finished: 13205
Finished: 13206
Finished: 13207
Finished: 13208
Finished: 13209
Finished: 13210
Finished: 13211
Finished: 13212
Finished: 13213
Finished: 13214
Finished: 13215
Finished: 13216
Finished: 13217
Finished: 13218
Finished: 13219
Finished: 13220
Finished: 13221
Finished: 13222
Finished: 13223
Finished

Finished: 13675
Finished: 13676
Finished: 13677
Finished: 13678
Finished: 13679
Finished: 13680
Finished: 13681
Finished: 13682
Finished: 13683
Finished: 13684
Finished: 13685
Finished: 13686
Finished: 13687
Finished: 13688
Finished: 13689
Finished: 13690
Finished: 13691
Finished: 13692
Finished: 13693
Finished: 13694
Finished: 13695
Finished: 13696
Finished: 13697
Finished: 13698
Finished: 13699
Finished: 13700
Finished: 13701
Finished: 13702
Finished: 13703
Finished: 13704
Finished: 13705
Finished: 13706
Finished: 13707
Finished: 13708
Finished: 13709
Finished: 13710
Finished: 13711
Finished: 13712
Finished: 13713
Finished: 13714
Finished: 13715
Finished: 13716
Finished: 13717
Finished: 13718
Finished: 13719
Finished: 13720
Finished: 13721
Finished: 13722
Finished: 13723
Finished: 13724
Finished: 13725
Finished: 13726
Finished: 13727
Finished: 13728
Finished: 13729
Finished: 13730
Finished: 13731
Finished: 13732
Finished: 13733
Finished: 13734
Finished: 13735
Finished: 13736
Finished

In [23]:
companys=pd.DataFrame({"company_ids":pd.Series(company_ids[0:len(ba_scores)]),"company_urls":pd.Series(company_urls),"company_names":pd.Series(company_names),
                       "location1s":pd.Series(location1s),"location2s":pd.Series(location2s),
                       "ba_scores":pd.Series(ba_scores),"ba_levels":pd.Series(ba_levels),"total_ratings":pd.Series(total_ratings),
                       "beer_currents":pd.Series(beer_currents),"total_reviews":pd.Series(total_reviews)
                      })

companys.to_csv("company.csv",encoding="utf-8")

In [ ]:
# TO DO: check accuracy of company info

In [ ]:
### Beer Page
# Ranking 
# Bros Score
# Availability
# Alc

# loop through rating (div id="rating_fullview_container)
    # 3.66/5  rDev +8.9%
    # look: 4 | smell: 3.5 | taste: 3.75 | feel: 3.75 | overall: 3.5
    # text?
    # username/id/page

In [ ]:
#### User Page

# beers
# beer karma

